## 深度学习中的张量维度格式：(N, C, H, W) vs (N, H, W, C)

这是深度学习中 4D 张量（tensor）的维度排列方式，每个字母代表一个维度：

| 符号 | 含义 | 说明 |
|------|------|------|
| **N** | Batch size | 批次大小，一次处理多少张图片 |
| **C** | Channels | 通道数（RGB图片是3，特征图可能是768、1536等） |
| **H** | Height | 高度（像素数） |
| **W** | Width | 宽度（像素数） |

### 两种格式的区别

| 格式 | 名称 | 例子 | 主要使用者 |
|------|------|------|------------|
| `(N, C, H, W)` | **channels-first** | `(2, 768, 6, 6)` | PyTorch、大多数CNN |
| `(N, H, W, C)` | **channels-last** | `(2, 6, 6, 768)` | TensorFlow、Swin Transformer |

### 实际例子

假设你有一个 batch 包含 2 张 192×192 的 RGB 图片：
- channels-first: `(2, 3, 192, 192)` → 2张图, 3通道, 192高, 192宽
- channels-last: `(2, 192, 192, 3)` → 2张图, 192高, 192宽, 3通道

### Swinv2 与 fastai 的兼容性问题

在使用 fastai 的 `vision_learner` 训练 swinv2 模型时会遇到错误：

```
RuntimeError: running_mean should contain 12 elements not 3072
```

**原因**：
- `convnext` 输出: `(2, 768, 6, 6)` ← channels-first ✅
- `swinv2` 输出: `(2, 6, 6, 1536)` ← channels-last ❌

fastai 的 `AdaptiveConcatPool2d` 期望 channels-first 格式，所以把 swinv2 的 `(2, 6, 6, 1536)` 误解为 C=6，导致了错误。

### 修复方法

添加 `permute(0, 3, 1, 2)` 将 `(N, H, W, C)` 转换为 `(N, C, H, W)`：

```python
# ========== 修复 Swin Transformer channels-last 输出问题 ==========
from fastai.vision.learner import TimmBody

_original_timm_body_forward = TimmBody.forward

def _patched_timm_body_forward(self, x):
    out = _original_timm_body_forward(self, x)
    # 检测 channels-last 格式并转换
    if out.dim() == 4 and out.shape[-1] > out.shape[1]:
        out = out.permute(0, 3, 1, 2)  # (N, H, W, C) -> (N, C, H, W)
    return out

TimmBody.forward = _patched_timm_body_forward
# ========== 修复结束 ==========
```

把这段代码放在 `from fastai.vision.all import *` 之后、`train` 函数定义之前即可。

## 决定 Batch Size 选择的真正因素

### 一个常见误解

> 既然可以通过**梯度累加（Gradient Accumulation）**解决显存不足的问题，那么显存大小就不再是限制 batch size 的因素了吗？

没错，梯度累加确实可以在显存有限的情况下"模拟"大 batch 训练。但这引出了一个更本质的问题：**决定 batch size 选择的真正因素是什么？**

---

### 影响 Batch Size 选择的关键因素

| 因素 | 说明 |
|------|------|
| **1. 收敛稳定性** | 小 batch 的梯度估计噪声大，训练曲线抖动明显；大 batch 的梯度更准确，收敛更平稳 |
| **2. 泛化性能** | 研究表明小 batch 训练可能有更好的泛化能力——噪声可以帮助模型跳出尖锐的局部最优，找到更平坦的最优解 |
| **3. 学习率配合** | 大 batch 通常需要更大的学习率（线性缩放法则：batch size 翻倍，lr 也翻倍），否则训练效果会变差 |
| **4. GPU 计算效率** | batch size 太小会导致 GPU 利用率低下——GPU 擅长并行计算，需要足够的数据才能"喂饱"它 |
| **5. BatchNorm 敏感性** | Batch Normalization 层依赖 batch 内的统计量，batch size 过小会导致统计不稳定，影响模型性能 |
| **6. 训练总时间** | 小 batch 需要更多迭代次数才能遍历完整数据集，可能增加总训练时间 |

---

### 梯度累加 vs 真正的大 Batch

虽然梯度累加可以实现等效的大 batch，但有一些细微差别：

```
真正的大 batch:  一次前向传播 64 个样本 → 计算梯度 → 更新参数
梯度累加:        8 次前向传播 × 8 个样本 → 累加梯度 → 更新参数
```

**主要区别**：
- **BatchNorm 行为不同**：每次前向传播只能看到当前 mini-batch 的统计量，而非完整的 64 个样本
- **时间效率**：梯度累加需要多次前向传播，总计算时间更长（但显存占用更小）

---

### 实践建议

1. **起点**：从能放进显存的最大 batch size 开始尝试
2. **调整**：观察训练曲线，如果太不稳定就增大 batch size，如果泛化差就减小
3. **配合**：调整 batch size 时同步调整学习率
4. **注意**：使用 BatchNorm 的模型，batch size 最好不要小于 16-32